In [ ]:
#%load_ext autoreload
#%autoreload 2

import numpy as np

from utilities.classes import *
from utilities.general import FPS
from utilities.plotting import plot_regression, plot_projection
from utilities.kernels import center_kernel
from matplotlib import pyplot as plt

from utilities.kernels import linear_kernel, gaussian_kernel

In [ ]:
training = np.load('../KernelPCovR/analysis/workspace/44510f9e5dec42b711c1467f03f7e504/training.npz')
testing = np.load('../KernelPCovR/analysis/workspace/44510f9e5dec42b711c1467f03f7e504/testing.npz')

In [ ]:
X = training["X"][:2000] * training['Xs'] + training['Xc']
Y = training["Y"][:2000] * training['Ys'] + training['Yc']
Xt = testing["X"][:2000] * testing['Xs'] + testing['Xc'] # Same as training
Yt = testing["Y"][:2000] * testing['Ys'] + testing['Yc'] # Same as training

fps, _ = FPS(X.T, 200)
X = X[:, fps]
Xt = Xt[:, fps]

# Mess with some shit
#X += np.random.uniform(-0.5, 0.5, X.shape[1])
#Y += np.random.uniform(-0.5, 0.5, Y.shape[1])

#Xt += np.random.uniform(-0.5, 0.5, Xt.shape[1])
#Yt += np.random.uniform(-0.5, 0.5, Yt.shape[1])

In [ ]:
scp = {"linewidth": 0.5, "edgecolor":"k", "cbar": False}

In [ ]:
pca_uncentered = PCA(n_PC=2, scale=False, center=False)
pca_uncentered.fit(X)

pca_centered = PCA(n_PC=2, scale=True, center=True)
pca_centered.fit(X)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_projection(Yt, pca_uncentered.transform(Xt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
plot_projection(Yt, pca_centered.transform(Xt), 
                fig=fig, title='Centered', ax=ax[1], **scp)

In [ ]:
lr_centered = LR(center=True, scale=True, regularization=1.0E-15)
lr_centered.fit(X, Y)
print('finished centered fit')

lr_uncentered = LR(center=False, scale=False, regularization=1.0E-15)
lr_uncentered.fit(X, Y)
print('finished uncentered fit')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_regression(Yt, lr_uncentered.transform(Xt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
print('done with uncentered')
plot_regression(Yt, lr_centered.transform(Xt), 
                fig=fig, title='Centered', ax=ax[1], **scp)
print('done with centered')

In [ ]:
#K = linear_kernel(X, X)
#Kt = linear_kernel(Xt, X)
K = gaussian_kernel(X, X)
Kt = gaussian_kernel(Xt, X)

In [ ]:
kpca_uncentered = KPCA(n_PC=2, kernel_type='linear', scale=False, center=False)
kpca_uncentered.fit(K=K)

kpca_centered = KPCA(n_PC=2, kernel_type='linear', scale=True, center=True)
kpca_centered.fit(K=K)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_projection(Yt, kpca_uncentered.transform(K=Kt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
plot_projection(Yt, kpca_centered.transform(K=Kt), 
                fig=fig, title='Centered', ax=ax[1], **scp)

In [ ]:
krr_centered = KRR(kernel_type='linear', center=True, scale=True)
krr_centered.fit(K=K, Y=Y)

krr_uncentered = KRR(kernel_type='linear', center=False, scale=False)
krr_uncentered.fit(K=K, Y=Y)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_regression(Yt, krr_uncentered.transform(K=Kt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
plot_regression(Yt, krr_centered.transform(K=Kt), 
                fig=fig, title='Centered', ax=ax[1], **scp)

In [ ]:
n_active = 50
fps_sparse, _ = FPS(X, n_active)
X_sparse = X[fps_sparse]

In [ ]:
#Knm = linear_kernel(X, X_sparse)
#Knmt = linear_kernel(Xt, X_sparse)
#Kmm = linear_kernel(X_sparse, X_sparse)
Knm = gaussian_kernel(X, X_sparse)
Knmt = gaussian_kernel(Xt, X_sparse)
Kmm = gaussian_kernel(X_sparse, X_sparse)

In [ ]:
skpca_uncentered = SparseKPCA(kernel_type='linear', n_PC=2, n_active=n_active,
                              scale=False, center=False)
skpca_uncentered.fit(X=X, Knm=Knm, Kmm=Kmm)

skpca_centered = SparseKPCA(kernel_type='linear', n_PC=2, n_active=n_active,
                            scale=True, center=True)
skpca_centered.fit(X=X, Knm=Knm, Kmm=Kmm)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_projection(Yt, skpca_uncentered.transform(X=Xt, Knm=Knmt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
plot_projection(Yt, skpca_centered.transform(X=Xt, Knm=Knmt), 
                fig=fig, title='Centered', ax=ax[1], **scp)

In [ ]:
skrr_centered = SparseKRR(kernel_type='linear', n_active=n_active,center=True, scale=True)
skrr_centered.fit(X=X, Knm=Knm, Kmm=Kmm, Y=Y)

skrr_uncentered = SparseKRR(kernel_type='linear', n_active=n_active, center=False, scale=False)
skrr_uncentered.fit(X=X, Knm=Knm, Kmm=Kmm, Y=Y)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
plot_regression(Yt, skrr_uncentered.transform(X=Xt, Knm=Knmt), 
                fig=fig, title='Uncentered', ax=ax[0], **scp)
plot_regression(Yt, skrr_centered.transform(X=Xt, Knm=Knmt), 
                fig=fig, title='Centered', ax=ax[1], **scp)

In [ ]:
alpha = 1.0

In [ ]:
PCovR_uncentered = PCovR(alpha=alpha, n_PC=2, scale=False, center=False)
PCovR_uncentered.fit(X, Y)

PCovR_centered = PCovR(alpha=alpha, n_PC=2, scale=True, center=True)
PCovR_centered.fit(X, Y)

In [ ]:
T_unc, Yp_unc, Xr_unc = PCovR_uncentered.transform(Xt)
T_c, Yp_c, Xr_c = PCovR_centered.transform(Xt)


fig, ax = plt.subplots(2,2, figsize=(12,12))
plot_projection(Yt, T_unc, 
                fig=fig, title='Uncentered', ax=ax[0][0], **scp)
plot_projection(Yt, T_c,
                fig=fig, title='Centered', ax=ax[0][1], **scp)
plot_regression(Yt, Yp_unc, 
                fig=fig, ax=ax[1][0], **scp)
plot_regression(Yt, Yp_c,
                fig=fig, ax=ax[1][1], **scp)

In [ ]:
KPCovR_uncentered = KPCovR(kernel_type='linear', alpha=alpha, n_PC=2, scale=False, center=False, regularization=1.0E-15)
KPCovR_uncentered.fit(X, Y, K=K)

KPCovR_centered = KPCovR(kernel_type='linear', alpha=alpha, n_PC=2, scale=True, center=True)
KPCovR_centered.fit(X, Y, K=K)

In [ ]:
T_unc, Yp_unc, Xr_unc = KPCovR_uncentered.transform(K=Kt)
T_c, Yp_c, Xr_c = KPCovR_centered.transform(K=Kt)


fig, ax = plt.subplots(2,2, figsize=(12,12))
plot_projection(Yt, T_unc, 
                fig=fig, title='Uncentered', ax=ax[0][0], **scp)
plot_projection(Yt, T_c,
                fig=fig, title='Centered', ax=ax[0][1], **scp)
plot_regression(Yt, Yp_unc, 
                fig=fig, ax=ax[1][0], **scp)
plot_regression(Yt, Yp_c,
                fig=fig, ax=ax[1][1], **scp)

In [ ]:
skpcovr_uncentered = SparseKPCovR(kernel_type='linear', alpha=alpha, n_PC=2, 
                                 n_active=n_active,
                                 scale=False, center=False)
skpcovr_uncentered.fit(X, Y, Knm=Knm, Kmm=Kmm)

skpcovr_centered = SparseKPCovR(kernel_type='linear', alpha=alpha, n_PC=2, 
                                 n_active=n_active,
                               scale=True, center=True)
skpcovr_centered.fit(X, Y, Knm=Knm, Kmm=Kmm)

In [ ]:
T_unc, Yp_unc, Xr_unc = skpcovr_uncentered.transform(X=Xt, Knm=Knmt)
T_c, Yp_c, Xr_c = skpcovr_centered.transform(X=Xt, Knm=Knmt)


fig, ax = plt.subplots(2,2, figsize=(12,12))
plot_projection(Yt, T_unc, 
                fig=fig, title='Uncentered', ax=ax[0][0], **scp)
plot_projection(Yt, T_c,
                fig=fig, title='Centered', ax=ax[0][1], **scp)
plot_regression(Yt, Yp_unc, 
                fig=fig, ax=ax[1][0], **scp)
plot_regression(Yt, Yp_c,
                fig=fig, ax=ax[1][1], **scp)

In [ ]:
# Check centering of Kmm
Knm = gaussian_kernel(X, X_sparse)
Kmnt = gaussian_kernel(Xt, X_sparse)
Kmm = gaussian_kernel(X_sparse, X_sparse)
Kmmc = center_kernel(Kmm)/1.0E5
Kmmcc = Kmm - np.mean(Knm, axis=0)

In [ ]:
print(np.linalg.pinv(Kmm))
print(np.linalg.pinv(Kmmc))

In [ ]:
print(np.linalg.eigvalsh(Kmm))
print(np.linalg.eigvalsh(Kmmc))

In [ ]:
skpca_uncentered_Kmm = SparseKPCA(kernel_type='linear', n_PC=2, n_active=n_active,
                              scale=True, center=True)
skpca_uncentered_Kmm.fit(X=X, Knm=Knm, Kmm=Kmm)

skpca_centered_Kmm = SparseKPCA(kernel_type='linear', n_PC=2, n_active=n_active,
                            scale=True, center=True)
skpca_centered_Kmm.fit(X=X, Knm=Knm, Kmm=Kmmc)

In [ ]:
print(skpca_uncentered_Kmm.transform(X=Xt, Knm=Knmt))
print(skpca_centered_Kmm.transform(X=Xt, Knm=Knmt))

In [ ]:
skrr_uncentered_Kmm = SparseKRR(kernel_type='linear', n_active=n_active, center=True, scale=True)
skrr_uncentered_Kmm.fit(X=X, Knm=Knm, Kmm=Kmm, Y=Y)

skrr_centered_Kmm = SparseKRR(kernel_type='linear', n_active=n_active,center=True, scale=True)
skrr_centered_Kmm.fit(X=X, Knm=Knm, Kmm=Kmmc, Y=Y)

In [ ]:
print(skrr_uncentered_Kmm.transform(X=Xt, Knm=Knmt))
print(skrr_centered_Kmm.transform(X=Xt, Knm=Knmt))

In [ ]:
skpcovr_uncentered_Kmm = SparseKPCovR(kernel_type='linear', alpha=1.0, n_PC=2, 
                                 n_active=n_active,
                                 scale=True, center=True)
skpcovr_uncentered_Kmm.fit(X, Y, Knm=Knm, Kmm=Kmm)

skpcovr_centered_Kmm = SparseKPCovR(kernel_type='linear', alpha=1.0, n_PC=2, 
                                 n_active=n_active,
                               scale=True, center=True)
skpcovr_centered_Kmm.fit(X, Y, Knm=Knm, Kmm=Kmmc)

T_unc, Yp_unc, Xr_unc = skpcovr_uncentered_Kmm.transform(X=Xt, Knm=Knmt)
T_c, Yp_c, Xr_c = skpcovr_centered_Kmm.transform(X=Xt, Knm=Knmt)

In [ ]:
print(T_unc)
print(T_c)

In [ ]:
print(Yp_unc)
print(Yp_c)

In [ ]:
print(Xr_unc)
print(Xr_c)